In [14]:
# Import libraries
import requests
import pandas as pd
import os
import duckdb
from datetime import datetime
from dotenv import load_dotenv
from pathlib import Path

load_dotenv('.env')

# Setting pandas to display all columns and rows
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

True

In [15]:
# Load API key from .env file
api_key = os.getenv('API_KEY')
print(api_key)

bca2413f10b586fc8e29e6a7712258baf2d528cfdfc06dcba5bf1cc44e714a9c


In [16]:
# Load API key from .env file
api_key = os.getenv('API_KEY')

# Create function to call API
def extract(api_key):
    """
    Fetch and process air quality measurements from the OpenAQ API.

    Parameters
    ----------
    api_key : str
        API key for authentication with the OpenAQ API.

    Returns
    -------
    df : pandas.DataFrame
        Dataframe containing the fetched measurements.
        
    Raises
    ------
    requests.exceptions.HTTPError
        For HTTP related errors.
    requests.exceptions.ConnectionError
        For connection related errors.
    requests.exceptions.Timeout
        For timeout errors.
    requests.exceptions.RequestException
        For all other request related errors.
    Exception
        If the fetched dataframe is empty.

    """
    
    # API URL
    api_url = "https://api.openaq.org/v2/measurements"
    
    # Define the query parameters to API
    params = {
        "location_id": "380422",
        "parameter": ["pressure", "temperature", "um003", "um025", "um010", "pm10", "um100", "pm1", "um005", "humidity", "um050", "pm25"],
        "limit": 1000,
        "api_key": api_key
    }
    
    try:
        # Make the GET request
        response = requests.get(api_url, params=params, timeout=30)
        
        # Raise exception for HTTP errors
        response.raise_for_status()
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            output = pd.json_normalize(data['results'])
            df = pd.DataFrame(output)
            
            # Check if dataframe is empty
            if df.empty:
                raise Exception("Empty df, check API request")
            
            df['date.utc'] = pd.to_datetime(df['date.utc'], errors='coerce') # Convert to datetime
            df['date.local'] = df['date.utc'].dt.tz_convert('America/Los_Angeles') # Covert to PST/PDT time zone
            df['date.local'] = df['date.local'].dt.tz_localize(None) # Convert to timezone-naive
            df = df[df['value'] > 0.0] # Filter values
            
            return df
        
    except requests.exceptions.HTTPError as errh:
        print("Http Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("Error: Something Else", err)
    except Exception as e:
        print(e)

In [17]:
# Call extract function
df = extract(api_key)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 913 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   locationId             913 non-null    int64              
 1   location               913 non-null    object             
 2   parameter              913 non-null    object             
 3   value                  913 non-null    float64            
 4   unit                   913 non-null    object             
 5   country                913 non-null    object             
 6   city                   0 non-null      object             
 7   isMobile               913 non-null    bool               
 8   isAnalysis             0 non-null      object             
 9   entity                 913 non-null    object             
 10  sensorType             913 non-null    object             
 11  date.utc               913 non-null    datetime64[ns, UTC]
 12 

In [18]:
# Define current working directory
current_dir = os.getcwd()
print(current_dir)

# Get parent directory
parent_directory = os.path.dirname(current_dir)
print(parent_directory)

# Define directory to store DuckDB database
database_directory = os.path.join(parent_directory, "data")
print(database_directory)

# # Create data directory to store DuckDB database
# Path(database_directory).mkdir(parents=True, exist_ok=True)

c:\Users\Ale\Documents\Data-Science-Projects\Ploomber-2023\ETL-pipeline-ML\src\notebooks
c:\Users\Ale\Documents\Data-Science-Projects\Ploomber-2023\ETL-pipeline-ML\src
c:\Users\Ale\Documents\Data-Science-Projects\Ploomber-2023\ETL-pipeline-ML\src\data


In [19]:
# Define table name
table_name = "air_data"

# Create local DuckDB instance
def init_duckdb(df, table_name, database_directory):
    """
    Initiate a DuckDB instance to create a DuckDB database named "air_data.duckdb" inside the
    given `database_directory` if it does not already exist. If the specified `table_name`
    does not exist in the database, it will be created and the given DataFrame `df`
    will be stored in it.

    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to be registered and stored in the DuckDB database.
    table_name : str
        Name of the table in which the DataFrame should be stored.
    database_directory : str
        Directory path where the DuckDB database file ("air_data.duckdb") will be located.

    Returns
    -------
    None

    """
    duckdb_directory = os.path.join(database_directory, "air_data.duckdb")
    con = duckdb.connect(duckdb_directory)
    con.register('df', df)
    
    # Check if table already exists, if not, create it
    tables = con.execute("SHOW TABLES").fetchall()
    if table_name not in [table[0] for table in tables]:
        con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")
    
    con.close()

In [20]:
# Call init_duckdb function
init_duckdb(df, table_name, database_directory)

In [21]:
df.head()

,locationId,location,parameter,value,unit,country,city,isMobile,isAnalysis,entity,sensorType,date.utc,date.local,coordinates.latitude,coordinates.longitude
0,380422,Tipperary Park,pressure,999.221333,mb,CA,None,False,None,Community Organization,low-cost sensor,2023-10-25 00:00:00+00:00,2023-10-24 17:00:00,49.208733,-122.9118
1,380422,Tipperary Park,um003,1.583667,particles/cm³,CA,None,False,None,Community Organization,low-cost sensor,2023-10-25 00:00:00+00:00,2023-10-24 17:00:00,49.208733,-122.9118
2,380422,Tipperary Park,pm1,0.070000,µg/m³,CA,None,False,None,Community Organization,low-cost sensor,2023-10-25 00:00:00+00:00,2023-10-24 17:00:00,49.208733,-122.9118
3,380422,Tipperary Park,um010,0.050667,particles/cm³,CA,None,False,None,Community Organization,low-cost sensor,2023-10-25 00:00:00+00:00,2023-10-24 17:00:00,49.208733,-122.9118
4,380422,Tipperary Park,temperature,47.833333,f,CA,None,False,None,Community Organization,low-cost sensor,2023-10-25 00:00:00+00:00,2023-10-24 17:00:00,49.208733,-122.9118


In [22]:
# Get MotherDuck token
md_token = os.getenv('MOTHERDUCK_TOKEN')

# Check if MotherDuck token is set into the environment
assert md_token and md_token.strip() != '', "MOTHERDUCK_TOKEN environment variable is not set or is empty."

# # Define database directory
# duckdb_directory = os.path.join(database_directory, "air_data.duckdb")

# Initiate the MotherDuck connection with token
local_con = duckdb.connect(f'md:?motherduck_token={md_token}')

# Load MotherDuck extension
local_con.execute("LOAD motherduck")

# Load air_data.duckdb database into MotherDuck database named openaq_api
local_con.execute("CREATE OR REPLACE TABLE openaq_api.main.df as SELECT * FROM 'df'")


In [23]:
air_df = local_con.sql("SELECT * FROM df").df()
air_df.head()

,locationId,location,parameter,value,unit,country,city,isMobile,isAnalysis,entity,sensorType,date.utc,date.local,coordinates.latitude,coordinates.longitude
0,380422,Tipperary Park,pressure,999.221333,mb,CA,NaN,False,NaN,Community Organization,low-cost sensor,2023-10-24 17:00:00-07:00,2023-10-24 17:00:00,49.208733,-122.9118
1,380422,Tipperary Park,um003,1.583667,particles/cm³,CA,NaN,False,NaN,Community Organization,low-cost sensor,2023-10-24 17:00:00-07:00,2023-10-24 17:00:00,49.208733,-122.9118
2,380422,Tipperary Park,pm1,0.070000,µg/m³,CA,NaN,False,NaN,Community Organization,low-cost sensor,2023-10-24 17:00:00-07:00,2023-10-24 17:00:00,49.208733,-122.9118
3,380422,Tipperary Park,um010,0.050667,particles/cm³,CA,NaN,False,NaN,Community Organization,low-cost sensor,2023-10-24 17:00:00-07:00,2023-10-24 17:00:00,49.208733,-122.9118
4,380422,Tipperary Park,temperature,47.833333,f,CA,NaN,False,NaN,Community Organization,low-cost sensor,2023-10-24 17:00:00-07:00,2023-10-24 17:00:00,49.208733,-122.9118


In [24]:
# duckdb_df = duckdb.sql("SELECT * FROM df").df()
# duckdb_df.head()